In [28]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.utils import shuffle
import numpy as np

In [4]:
games = pd.read_csv('finalExplodedChess.csv')
games.head(10)


,00,01,02,03,04,05,06,07,10,11,...,opening,pawnLines,pawnRanks,pawnsGuardingKings,pins,protects,stackedPawns,victory_type,white_rating,winner
0,5,3,4,1000,9,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,2,0,0,0,0,victory_type,1659,white
1,5,3,4,1000,9,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,0,0,0,-4,0,victory_type,1659,white
2,5,3,0,1000,9,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,0,0,0,0,0,victory_type,1659,white
3,5,3,0,1000,9,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,0,0,0,-1,0,victory_type,1659,white
4,5,3,0,1000,0,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,0,0,0,0,0,victory_type,1659,white
5,5,3,0,1000,0,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,-1,0,-2,-4,0,victory_type,1659,white
6,5,3,0,0,0,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,-1,-2,-2,-5,0,victory_type,1659,white
7,5,3,0,0,0,4,3,5,1,1,...,Bishop's Opening: Boi Variation,0,-1,-2,-2,-3,0,victory_type,1659,white
8,5,3,0,0,0,4,0,5,1,1,...,Bishop's Opening: Boi Variation,0,-1,-2,-2,-4,0,victory_type,1659,white
9,5,3,0,0,0,4,0,5,1,1,...,Bishop's Opening: Boi Variation,0,-1,-2,-2,-1,0,victory_type,1659,white


In [5]:
games.dtypes

00                     int64
01                     int64
02                     int64
03                     int64
04                     int64
05                     int64
06                     int64
07                     int64
10                     int64
11                     int64
12                     int64
13                     int64
14                     int64
15                     int64
16                     int64
17                     int64
20                     int64
21                     int64
22                     int64
23                     int64
24                     int64
25                     int64
26                     int64
27                     int64
30                     int64
31                     int64
32                     int64
33                     int64
34                     int64
35                     int64
                       ...  
72                     int64
73                     int64
74                     int64
75            

In [6]:
games.groupby('winner').count()

,00,01,02,03,04,05,06,07,10,11,...,moves,opening,pawnLines,pawnRanks,pawnsGuardingKings,pins,protects,stackedPawns,victory_type,white_rating
winner,,,,,,,,,,,,,,,,,,,,,
black,536417,536417,536417,536417,536417,536417,536417,536417,536417,536417,...,536417,536417,536417,536417,536417,536417,536417,536417,536417,536417
white,561202,561202,561202,561202,561202,561202,561202,561202,561202,561202,...,561202,561202,561202,561202,561202,561202,561202,561202,561202,561202


In [7]:
games['white_win'] = games['winner']

In [8]:
games['white_win'] = games['white_win'].replace('white', 1)
games['white_win'] = games['white_win'].replace('black', 0)

In [9]:
columns = ['basicScore', 'canCastle', 'captures', 'centrePawns', 'checked', 'enpassants', 'fianachettos', 'forks', 'hasCastled', 'kingMoves', 'move', 'moves', 'pawnLines', 'pawnRanks', 'pawnsGuardingKings', 'pins', 'protects', 'stackedPawns', 'white_win']

In [10]:
df = games[columns]

In [11]:
df.dtypes

basicScore            int64
canCastle             int64
captures              int64
centrePawns           int64
checked               int64
enpassants            int64
fianachettos          int64
forks                 int64
hasCastled            int64
kingMoves             int64
move                  int64
moves                 int64
pawnLines             int64
pawnRanks             int64
pawnsGuardingKings    int64
pins                  int64
protects              int64
stackedPawns          int64
white_win             int64
dtype: object

In [12]:
df['Intercept'] = 1.0

/home/eoin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
X = df[columns]
X = X.drop('white_win', axis=1)

In [23]:
y = df['white_win']

In [24]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
basicScore,1097619.0,0.057066,4.343276,-46.0,-1.0,0.0,1.0,66.0
canCastle,1097619.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
captures,1097619.0,-2.427586,202.713712,-4024.0,-2.0,0.0,2.0,5007.0
centrePawns,1097619.0,-0.551261,0.747044,-4.0,-1.0,0.0,0.0,2.0
checked,1097619.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
enpassants,1097619.0,0.015018,0.557884,-2.0,0.0,0.0,0.0,2.0
fianachettos,1097619.0,-0.045017,0.414149,-2.0,0.0,0.0,0.0,2.0
forks,1097619.0,-0.820214,133.804081,-2010.0,0.0,0.0,0.0,2011.0
hasCastled,1097619.0,0.053117,0.505376,-1.0,0.0,0.0,0.0,1.0
kingMoves,1097619.0,0.128587,1.785991,-8.0,-1.0,0.0,1.0,8.0


In [29]:
f1 = cross_val_score(LogisticRegression(), X, y, scoring='f1', cv=5)
f1 = np.append(f1, f1.mean())

accuracy = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=5)
accuracy = np.append(accuracy, accuracy.mean())

precision = cross_val_score(LogisticRegression(), X, y, scoring='precision', cv=5)
precision = np.append(precision, precision.mean())

recall = cross_val_score(LogisticRegression(), X, y, scoring='recall', cv=5)
recall = np.append(recall, recall.mean())

df_cv5_results = pd.DataFrame({'Test':['1st', '2nd', '3rd', '4th', '5th', 'Averages'],'F1':f1, 'Accuracy':accuracy, 'Precision':precision, 'Recall':recall})
df_cv5_results[['Test', 'Accuracy', 'Precision', 'Recall', 'F1']].set_index('Test')

,Accuracy,Precision,Recall,F1
Test,,,,
1st,0.667022,0.658480,0.724521,0.689924
2nd,0.666954,0.657736,0.726838,0.690563
3rd,0.669183,0.660438,0.726506,0.691898
4th,0.666819,0.657870,0.725820,0.690176
5th,0.667570,0.658009,0.728395,0.691415
Averages,0.667509,0.658507,0.726416,0.690795
